<a href="https://colab.research.google.com/github/jaivanti/syracuse/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEMO1


Import Libraries 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install opencv-contrib-python==4.3.0.36



In [ ]:
!pip install pafy


In [ ]:
!pip install imutils pafy youtube-dl

In [ ]:
import cv2
from cv2 import dnn_superres
import numpy as np
import imutils


In [ ]:
import tensorflow as tf
import os,sys
import PIL
import os
from PIL import Image

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

Make a directory 'PATH'

In [ ]:
!mkdir -p /content/PATH/
%cd PATH

[Errno 2] No such file or directory: 'PATH'
/content/PATH


Convert a youtube video  into Frames

In [ ]:
import pafy
url = 'https://www.youtube.com/watch?v=FtdpDjV0dMY' #Insert a youtube link
video = pafy.new(url)
videoStream = video.getbestvideo()
vidcap  = cv2.VideoCapture(videoStream.url)

In [ ]:
def getFrame(time,frameCount):
   vidcap.set(cv2.CAP_PROP_POS_MSEC,time)
   frameDet, frame = vidcap.read()
   if frameDet:
      frame = imutils.resize(frame,width=128)
      cv2.imwrite(str(frameCount)+".jpg", frame)

In [ ]:
startTime = 0
frameRate = 500
frameCount = 1
h,m,s = video.duration.split(sep=':')
endTime = (int(h)*3600 + int(m)*60 +int(s))*1000
for time in range(startTime,endTime,frameRate):
   frameCount = frameCount + 1
   time = round(time,2)
   getFrame(time,frameCount)

Download Pretrained model and start Processing for SR

In [ ]:
# Create an SR object
sr = dnn_superres.DnnSuperResImpl_create()


In [ ]:
path = "/content/drive/MyDrive/SR CDN/ESPCN_x3.pb" #(suyash path)
#path = "/content/drive/MyDrive/ESPCN_x3.pb" #(jaivanti path)
sr.readModel(path)

In [ ]:
sr.setModel("espcn",3) #upscale factor=3

In [ ]:
directory = '/content/PATH/'
for filename in os.scandir(directory):
    if (filename.path.endswith(".jpg") or filename.path.endswith(".png")):
        print(filename.path)
        image = cv2.imread(filename.path)
        result = sr.upsample(image)
        cv2.imwrite("./upscaled.png", result)
        

/content/PATH/7.jpg
/content/PATH/2.jpg
/content/PATH/5.jpg
/content/PATH/3.jpg
/content/PATH/upscaled.png
/content/PATH/6.jpg
/content/PATH/4.jpg


INFERENCE TIME CALCULATION(PER FRAME)

In [ ]:
import time
directory = '/content/PATH/'
#start = time.time()
for filename in os.scandir(directory):
    if (filename.path.endswith(".jpg") or filename.path.endswith(".png")):
        print(filename.path)
        image = cv2.imread(filename.path)
        start = time.time()
        result = sr.upsample(image)
        end = time.time()
        #cv2.imwrite("./upscaled.png", result)
        print("[INFO] inference time {:.6f} seconds ".format(end - start)) 

/content/PATH/7.jpg
[INFO] inference time 0.058319 seconds 
/content/PATH/2.jpg
[INFO] inference time 0.051571 seconds 
/content/PATH/5.jpg
[INFO] inference time 0.052205 seconds 
/content/PATH/3.jpg
[INFO] inference time 0.058965 seconds 
/content/PATH/upscaled.png
[INFO] inference time 0.483574 seconds 
/content/PATH/6.jpg
[INFO] inference time 0.069253 seconds 
/content/PATH/4.jpg
[INFO] inference time 0.053618 seconds 


UPSCALING THE FRAMES

In [ ]:
#upscaling
for i in range(2,8):
  name = '/content/PATH/' + str(i) + '.jpg'
  image = cv2.imread(name)
  result = sr.upsample(image)
  name_f = '/content/PATH/up' + str(i) + '.png'
  cv2.imwrite(name_f, result)

RESIZING

In [ ]:
#RESIzING
for i in range(2,8):
  img = cv2.imread('/content/PATH/' + str(i) + '.jpg')
  
  scale_percent = 300 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
    
  # resize image
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

  name = '/content/PATH/resized' + str(i) + '.png'
  cv2.imwrite(name,resized )
  
  
  #print('Resized Dimensions : ',resized.shape)
  
  #cv2_imshow(resized)
  cv2.waitKey(0)
  cv2.destroyAllWindows()


PSNR CALCULATION

In [ ]:
from math import log10, sqrt
import cv2
import numpy as np

def PSNR(original, compressed):
	mse = np.mean((original - compressed) ** 2)
	if(mse == 0): # MSE is zero means no noise is present in the signal .
				# Therefore PSNR have no importance.
		return 100
	max_pixel = 255.0
	psnr = 20 * log10(max_pixel / sqrt(mse))
	return psnr

In [ ]:
#PSNR
for i in range(2,8):
  name_c = '/content/PATH/resized' + str(i) + '.png'
  compressed = cv2.imread(name_c)
  name_o = '/content/PATH/Up' + str(i) + '.png'
  original = cv2.imread(name_o)
  value = PSNR(original, compressed)
  print(f"PSNR value is {value} dB")

SSIM CALCULATION

In [ ]:
#SSIM
from skimage.metrics import structural_similarity as ssim
for i in range(2,8):
  name_c = '/content/PATH/resized' + str(i) + '.png'
  compressed = cv2.imread(name_c)
  name_o = '/content/PATH/Up' + str(i) + '.png'
  original = cv2.imread(name_o)
  value = ssim(original, compressed, multichannel=True)
  print(f"ssim value is {value} dB")

MERGING FRAMES TO MAKE VIDEO

In [ ]:
#MERGING FRAMES
import cv2
import numpy as np
import glob

img_array = []
for i in range(2,8):
  name = '/content/PATH/Up' + str(i) + '.png'

  for filename in glob.glob(name):
      img = cv2.imread(filename)
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
